In [2]:
import leafmap
import geopandas as gpd
from geospatial_tools import DATA_DIR

## Base data

The USA polygon is base off 2018's `cb_2018_us_nation_5m` shapefile, taken from here: 
https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

It was then processed using QGIS to keep only the contiguous states, without any islands.

The Sentinel 2 grid was taken from the kml file found here: 
https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/data-products

It was then processed using QGIS to keep only the grid cells that overlap with the 
contiguous states, meaning the polygon layer which is described just above.

Since our area of study is quite large, the `EPSG:5070` projection was chosen, as it
covers the whole area, introduces minimal distortion while preserving area.

In [3]:
USA_POLYGON_FILE = DATA_DIR / "usa/usa_polygon_5070.gpkg"
S2_USA_GRID_FILE = DATA_DIR / "usa/s2_grid_usa_polygon_5070.gpkg"

In [27]:
usa_polygon = gpd.read_file(USA_POLYGON_FILE)
s2_grid = gpd.read_file(S2_USA_GRID_FILE)

In [28]:
usa_polygon

,AFFGEOID,GEOID,NAME,geometry
0,0100000US,US,United States,"MULTIPOLYGON (((-2123555.702 3120381.564, -212..."


In [29]:
s2_grid

,name,folders,description,altitude,alt_mode,time_begin,time_end,time_when,geometry
0,12TUP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1386334.944 2487548.770 0.0...
1,12TYQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-976300.478 2523767.452 0.00...
2,12TYR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-960099.705 2622374.255 0.00...
3,12TYN,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1008622.024 2325748.358 0.0...
4,12TYP,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-992478.385 2424861.340 0.00...
...,...,...,...,...,...,...,...,...,...
977,12TTM,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1515431.586 2304192.826 0.0...
978,12TUK,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1448525.813 2089886.667 0.0...
979,12TUQ,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1371006.917 2586590.133 0.0...
980,12TUR,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,,,,,MULTIPOLYGON Z (((-1355793.563 2685354.080 0.0...


In [30]:
m = leafmap.Map(center=[40, -98], zoom=4)

# In blue, the USA polygon
m.add_gdf(usa_polygon, layer='usa')
# In red, the Sentinel 2 grid
m.add_gdf(s2_grid, layer='s2_grid', style={"color": "red"})

m

Map(center=[40, -98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

## Creating our inference grid

From this, we want to create a grid of square polygons with which we will later on
query the [Planetary Computer](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a)
Sentinel 2 dataset and clip the selected Sentinel 2 images.

In [4]:
import time
from geospatial_tools.vector import create_vector_grid_parallel, to_geopackage, select_polygons_by_location
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor

In [32]:
grid_size = 5000
bbox = usa_polygon.total_bounds

In [ ]:
start = time.time()
print("Starting processing for [create_vector_grid_parallel]")
grid_parallel = create_vector_grid_parallel(bounding_box=bbox, grid_size=grid_size, crs="EPSG:5070")
stop = time.time()
print(f"Printing len(grid_parallel) to check if grid contains same amount of polygons : {len(grid_parallel)}")
print(f"Time taken to create parallel grid: {stop - start}")
to_geopackage(gdf=grid_parallel, filename="polygon_grid.gpkg")

Starting processing for [create_vector_grid_parallel]
[2024-06-03 16:00:29] INFO       [MainThread][geospatial_tools.vector] Creating grid coordinates for bounding box [[-2356113.74289801   310919.59963659  2258200.17691555  3165721.6501298 ]]
[2024-06-03 16:00:29] INFO       [MainThread][geospatial_tools.vector] Creating flattened grid coordinates
[2024-06-03 16:00:29] INFO       [MainThread][geospatial_tools.vector] Number of workers used: 16
[2024-06-03 16:00:29] INFO       [MainThread][geospatial_tools.vector] Allocating polygon array for [13175825] polygons
[2024-06-03 16:00:29] INFO       [MainThread][geospatial_tools.vector] Creating polygons from chunk


### Selecting the useful polygons

Now, since our grid was created using the extent of our input polygon (continental USA), we need to filter out the polygons that do not intersect with it.

Doing this in Python is not the most efficient way to do things, but since it's a step that shouldn't be done over and over, it's not that critical.

If ever you need to do this step in an efficient way because the data is just too big or too complex, it would be better off going through QGIS, PyGQIS, GDAL or 
some other more efficient way to do this operation. 

In [ ]:
start = time.time()
print("Starting intersect selection using for loop")
intersecting_polygons = select_polygons_by_location(grid_parallel, usa_polygon)
stop = time.time()
print(f"Time taken to intersect using for loop: {stop - start}")
# Optionally, save to a new file

### Visualizing the selected polygons

This will take more or less time, depending on the number on polygons. 

In [ ]:
m.add_gdf(intersecting_polygons, layer='intersecting_polygons', style={"color": "blue"})
m

In [1]:
%pip list | grep planetary

planetary-computer        1.0.0
Note: you may need to restart the kernel to use updated packages.


## Exploring S2 STAC catalog tools

In [5]:
from pystac_client import Client
import planetary_computer
from pathlib import Path
import requests
import datetime
from geospatial_tools import DATA_DIR
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

In [6]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=planetary_computer.sign_inplace,)
start_year = 2021 
end_year = 2023
date_ranges = []
for year in range(start_year, end_year + 1):
    june_start = datetime.datetime(year, 6, 1)
    july_end = datetime.datetime(year, 8, 1)
    date_ranges.append(f"{june_start.isoformat()}Z/{july_end.isoformat()}Z")
    
print(date_ranges)

results = []
tile_ids = ["10SGE", "10SGD"]
for date_range in date_ranges:
    print(date_range)
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        datetime=date_range,
        query={"eo:cloud_cover": {"lt": 5}, "s2:mgrs_tile": {"in": tile_ids}},
        sortby= [{"field": "properties.eo:cloud_cover", "direction": "asc"}]
    )
    results.extend(list(search.items()))


# (Optional) Further process the items 
for item in results:
    print(item.id, item.datetime, item.properties["eo:cloud_cover"]) 

['2021-06-01T00:00:00Z/2021-08-01T00:00:00Z', '2022-06-01T00:00:00Z/2022-08-01T00:00:00Z', '2023-06-01T00:00:00Z/2023-08-01T00:00:00Z']
2021-06-01T00:00:00Z/2021-08-01T00:00:00Z
2022-06-01T00:00:00Z/2022-08-01T00:00:00Z
2023-06-01T00:00:00Z/2023-08-01T00:00:00Z
S2B_MSIL2A_20210720T183919_R070_T10SGE_20210721T052505 2021-07-20 18:39:19.024000+00:00 0.202279
S2B_MSIL2A_20210713T184919_R113_T10SGE_20210714T084909 2021-07-13 18:49:19.024000+00:00 0.002955
S2B_MSIL2A_20210710T183919_R070_T10SGE_20210711T034836 2021-07-10 18:39:19.024000+00:00 0.237209
S2B_MSIL2A_20210630T183919_R070_T10SGE_20210701T140951 2021-06-30 18:39:19.024000+00:00 0.228867
S2B_MSIL2A_20210623T184919_R113_T10SGE_20210626T013807 2021-06-23 18:49:19.024000+00:00 0.008795
S2B_MSIL2A_20210620T183919_R070_T10SGE_20210621T095614 2021-06-20 18:39:19.024000+00:00 0.564451
S2B_MSIL2A_20210613T184919_R113_T10SGE_20210615T052420 2021-06-13 18:49:19.024000+00:00 0.017032
S2B_MSIL2A_20210603T184919_R113_T10SGE_20210604T100449 2021

In [9]:
image_ids = ["S2B_MSIL2A_20220615T183919_R070_T10SGE_20220618T191736", "S2B_MSIL2A_20220615T183919_R070_T10SGD_20220618T184146"]
file_base_path = Path(f"{DATA_DIR}/sentinel-2")
bands = ["B02", "B03", "B04", "B08", "visual"]

In [10]:
# Function to download an asset
def download_asset(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {filename} successfully.")
        return filename
    else:
        print(f"Failed to download the asset. Status code: {response.status_code}")
        return None

def reproject_raster(source_path, dataset_path, dataset_crs):
    with rasterio.open(source_path) as source:
        transform, width, height = calculate_default_transform(
            source.crs, dataset_crs, source.width, source.height, *source.bounds)
        kwargs = source.meta.copy()
        kwargs.update({
            'crs': dataset_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(dataset_path, 'w', **kwargs) as dataset:
            for i in range(1, source.count + 1):
                reproject(
                    source=rasterio.band(source, i),
                    destination=rasterio.band(dataset, i),
                    src_transform=source.transform,
                    src_crs=source.crs,
                    dst_transform=transform,
                    dst_crs=dataset_crs,
                    resampling=Resampling.nearest)
    print(f"Reprojected file created at {dataset_path}")

for item in results:
    image_id = item.id
    if image_id in image_ids:
        print(f"Title: {image_id}")
        print(f"Date: {item.datetime}")
        print(f"Assets: {list(item.assets.keys())}")
    
        downloaded_files = []

        for band in bands:
            if band in item.assets:
                asset = item.assets[band]
                asset_url = asset.href
                print(f"Downloading {band} from {asset_url}")

                file_name = file_base_path / f"{image_id}_{band}.tif"
                downloaded_file = download_asset(asset_url, file_name)
                if downloaded_file:
                    downloaded_files.append(downloaded_file)
            else:
                print(f"Band {band} not available for {image_id}.")

        if downloaded_files:
            merged_file = file_base_path / f"{image_id}_merged.tif"
            
            with rasterio.open(downloaded_files[0]) as source0:
                meta = source0.meta
                meta.update(count=len(downloaded_files))

            # TODO Fix for case of tif containing multiple bands
            with rasterio.open(merged_file, 'w', **meta) as merged_sentinel_image:
                for idx, file in enumerate(downloaded_files):
                    with rasterio.open(file) as sentinel_band_image:
                        merged_sentinel_image.write_band(idx + 1, sentinel_band_image.read(1))
                        merged_sentinel_image.set_band_description(idx + 1, sentinel_band_image.descriptions[0] if sentinel_band_image.descriptions[0] else bands[idx])
                        merged_sentinel_image.update_tags(idx + 1, **sentinel_band_image.tags(1))

            print(f"Merged file created at {merged_file}")

            reprojected_file = file_base_path / f"{image_id}_reprojected.tif"
            reproject_raster(merged_file, reprojected_file, 'EPSG:5070')

            for file in downloaded_files:
                os.remove(file)
            os.remove(merged_file)

Title: S2B_MSIL2A_20220615T183919_R070_T10SGE_20220618T191736
Date: 2022-06-15 18:39:19.024000+00:00
Assets: ['AOT', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'SCL', 'WVP', 'visual', 'preview', 'safe-manifest', 'granule-metadata', 'inspire-metadata', 'product-metadata', 'datastrip-metadata', 'tilejson', 'rendered_preview']
Downloaded /home/dev/projects/geospatial-tools/data/sentinel-2/S2B_MSIL2A_20220615T183919_R070_T10SGE_20220618T191736_B02.tif successfully.
Downloaded /home/dev/projects/geospatial-tools/data/sentinel-2/S2B_MSIL2A_20220615T183919_R070_T10SGE_20220618T191736_B03.tif successfully.
Downloaded /home/dev/projects/geospatial-tools/data/sentinel-2/S2B_MSIL2A_20220615T183919_R070_T10SGE_20220618T191736_B04.tif successfully.
Downloaded /home/dev/projects/geospatial-tools/data/sentinel-2/S2B_MSIL2A_20220615T183919_R070_T10SGE_20220618T191736_B08.tif successfully.
Downloaded /home/dev/projects/geospatial-tools/data/sentinel-2/S2B_MSIL2A

NameError: name 'rasterio' is not defined